In [1]:
import sys
import os

import numpy as np
import pandas as pd

#### Cases
- only the first entry record from the front-door
- the first entry record from other doors
- 

#### Potential Influence Factor
1. weekday (course on Satruday? or class room is still available on Saturday?)

A: No course on Weekend. But classrooms are still available

2. Jan. 14th (students still had courses?) 

A: Official there is no course before 14th Jan.

3. Other doors needed to login (What is the function of reception 1, 2, 3 

A: If someone wants to entry a classroom, he or she has to login from corresponding reception door  

4. What's the common form of courses (lesson + practice ？）

A: the same as in DSA

5. Is there any influence if students arrivate late, like 1 hour, or 2 hours?)
A: 

6. Is there some different of class content between the 2nd and 3rd

A: 3rd material, 2rd projects

7. Is there course twice weeks?

A: no

8. Is there course in the afternoon?

A: Sometimes might be a meetup

9. Is there exam day or project day in January?

A: Exam day:14th and 21th

10. During lunch time, where will student go?

A: Most of students re-logged after the lunch

11. How to deal with the "Even message" feature, especially when the "Access - door opened" comes?


#### Method for anomaly
- delete (Why?) (data lost)
    1. Data before 14th (not included) can be ignored (no course)
    2. weekday 5 (Saturday) / 6 (Sunday) can be ignored (no course)
    3. meetup events (10:30, 11:00, 14:00, 15:30, 18:00... ?) 
    4. remove those card numbers which appeared less than 5 days after 14th (meetup probably)
        - student would go out during the lunch break (10)
    5. remove those first log data after 12:00 everyday (10)
- backpropagate (how? depeneds on what)

#### Limitation
- Lack of students routine life background understanding (student schedule, student habits)
- Lack of understanding of log record anomaly (log record appeared around some particular time like 10:30)
- Data inadequate (less than an half month long data is useful)

In [2]:
"""
Configuration: Project (Analyzing Green Fox HQ logs)
"""
file_path = os.path.join(os.getcwd(), "data_sample", "logs.csv")
data_cols = ['Id', 'Date time', 'Event message', 'Event number', 'Object #1', 
             'Description #1', 'Object #2', 'Description #2', 'Object #3',
             'Description #3', 'Object #4', 'Description #4', 'Card number']

In [3]:
def read_csv_data(file_path, chunk_size= 5000, header= ''):
    df = pd.DataFrame([])
    for chunk in pd.read_csv(filepath_or_buffer= file_path, chunksize= chunk_size, header=header):
        df = df.append(chunk)
    return df

In [4]:
df_logs = read_csv_data(file_path, header= None)
df_logs.head()
df_logs.columns = data_cols

#### extract date and time

In [5]:
def get_week_day(start, end, freq= 'D'):
    """
    Output:
    0: Monday 1: Tuesday ... 6: Sunday
    """
    week_day = pd.date_range(start, end, freq= freq).to_series().dt.dayofweek
    return pd.DataFrame({'date': week_day.index.values, 'day': week_day.values})

def get_work_history(df_logs):
    return pd.DataFrame({'date' : df_logs['date'].value_counts().index.values, 
                         'people_count':df_logs['date'].value_counts().values})

def sort_by_date(df, date_col= 'date'):
    return df.sort_values(by= date_col).reset_index(drop= True)

def convert_datetime(df, date_col= 'date'):
    return pd.to_datetime(df[date_col])

In [6]:
df_logs['Date time'] = convert_datetime(df_logs, 'Date time')
df_logs['date'] = df_logs['Date time'].dt.strftime('%Y-%m-%d')
df_logs['time'] = df_logs['Date time'].dt.strftime('%H:%M:%S')

#### Get work history (date : people count)

In [7]:
df_work_history = get_work_history(df_logs)
df_work_history = sort_by_date(df_work_history, 'date')
df_work_history['date'] = convert_datetime(df_work_history, 'date')
df_work_history.head()

,date,people_count
0,2019-01-02,54
1,2019-01-03,85
2,2019-01-04,87
3,2019-01-05,20
4,2019-01-07,276


In [8]:
df_week_day = get_week_day('2019/01/01', '2019/01/31')
df_week_day['date'] = df_week_day['date'].dt.strftime('%Y-%m-%d')
df_week_day.head()

,date,day
0,2019-01-01,1
1,2019-01-02,2
2,2019-01-03,3
3,2019-01-04,4
4,2019-01-05,5


### Wrinkling data: df_logs 

In [9]:
# the amount of card number (people)
df_logs['Card number'].value_counts().shape

(194,)

In [10]:
df_logs['Event message'].value_counts()

Access granted          9705
Access - door opened     775
Name: Event message, dtype: int64

#### extract columns for df_log

In [13]:
sub_cols = np.array(['Date time', 'date', 'time', 'Description #1', 
                     'Card number', 'Description #2', 'Event message'])
doors = np.array(['A66 - 04 FÕBEJÁRAT (F-1) Door #1', 
                      'A66 - 12 Recepció (2-1) Door #1',
                      'A66 - 17 Recepció (3-1) Door #1',
                      'A66 - 06 Kávézó terasz (F-3) Door #1',
                      'A66 - 18 Elõtér 316 (3-2) Door #1'])

In [14]:
"""
There is no course officially before 14th Jan., hence here we could only take data 
after 14th (including) into consideration.
"""
df_after_14 = df_logs[df_logs['date'] >= '2019-01-14'][sub_cols]
# add a weekday feature
df_after_14 = pd.DataFrame(pd.merge(df_after_14, df_week_day, 
                                    on= 'date', how= 'left').reset_index(drop=True))
print(df_after_14.shape)
df_after_14.head()

(8649, 8)


,Date time,date,time,Description #1,Card number,Description #2,Event message,day
0,2019-01-14 07:58:17,2019-01-14,07:58:17,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00070:04181,Barczi Mihály,Access granted,0
1,2019-01-14 07:59:49,2019-01-14,07:59:49,A66 - 12 Recepció (2-1) Door #1,00070:04181,Barczi Mihály,Access granted,0
2,2019-01-14 08:26:32,2019-01-14,08:26:32,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00018:06288,Barish Nilofer Virág,Access granted,0
3,2019-01-14 08:27:21,2019-01-14,08:27:21,A66 - 12 Recepció (2-1) Door #1,00018:06288,Barish Nilofer Virág,Access granted,0
4,2019-01-14 08:27:48,2019-01-14,08:27:48,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00243:07476,Barta Gergõ,Access granted,0


In [15]:
# df_logs_after_14[df_logs_after_14['Event message'] == 'Access - door opened'][sub_cols]

In [16]:
# remove data on Saturday and Sunday
df_after_14 = df_after_14[~df_after_14['day'].isin([5, 6])]
df_after_14.shape

(8577, 8)

In [17]:
df_after_14['Description #1'].value_counts()

A66 - 04 FÕBEJÁRAT (F-1) Door #1          3484
A66 - 12 Recepció (2-1) Door #1           1899
A66 - 17 Recepció (3-1) Door #1           1720
A66 - 06 Kávézó terasz (F-3) Door #1       901
A66 - 18 Elõtér 316 (3-2) Door #1          495
A66 - 05 Vasrács (F-2) Door #1              39
A66 - 13 Elõtér 216 (2-2) Door #1           28
A66 - 02 Raktár (P-2) Door #1                8
A66 - 01 Kerékpár-tároló (P-1) Door #1       1
A66 - 22 Cselédlépcsõ Door                   1
A66 - 07 Recepció (1-1) Door #1              1
Name: Description #1, dtype: int64

In [18]:
# Extract the first one log record of each Card number
df_after14_first = df_after_14.drop_duplicates(subset=['Card number', 'date'], keep= 'first').reset_index(drop= True)
df_after14_first.head()

,Date time,date,time,Description #1,Card number,Description #2,Event message,day
0,2019-01-14 07:58:17,2019-01-14,07:58:17,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00070:04181,Barczi Mihály,Access granted,0
1,2019-01-14 08:26:32,2019-01-14,08:26:32,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00018:06288,Barish Nilofer Virág,Access granted,0
2,2019-01-14 08:27:48,2019-01-14,08:27:48,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00243:07476,Barta Gergõ,Access granted,0
3,2019-01-14 08:28:26,2019-01-14,08:28:26,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00178:59777,Takács Róbert,Access granted,0
4,2019-01-14 08:29:22,2019-01-14,08:29:22,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00161:44433,Salfay Anna,Access granted,0


In [19]:
df_after14_first['Card number'].value_counts().shape

(174,)

In [20]:
df_after14_first['Description #1'].value_counts()

A66 - 04 FÕBEJÁRAT (F-1) Door #1        1764
A66 - 12 Recepció (2-1) Door #1          100
A66 - 17 Recepció (3-1) Door #1           84
A66 - 06 Kávézó terasz (F-3) Door #1      33
A66 - 18 Elõtér 316 (3-2) Door #1         29
A66 - 05 Vasrács (F-2) Door #1             6
A66 - 02 Raktár (P-2) Door #1              1
A66 - 13 Elõtér 216 (2-2) Door #1          1
A66 - 07 Recepció (1-1) Door #1            1
Name: Description #1, dtype: int64

In [21]:
df_after14_first['Card number'].value_counts().shape

(174,)

In [22]:
# get card numbers whose frequencies are more than 5 after 14th Jan. 
people_usually = df_after14_first['Card number'].value_counts().index[pd.Series(df_after14_first['Card number'].value_counts() > 5)].values
len(people_usually)

148

In [23]:
df_regular_come = df_after_14[df_after_14['Card number'].isin(people_usually)]
df_regular_come.shape

(8343, 8)

In [24]:
df_regular_come_first = df_regular_come.drop_duplicates(subset=['Card number', 'date']).reset_index(drop=True)
df_regular_come_first['Description #1'].value_counts()

A66 - 04 FÕBEJÁRAT (F-1) Door #1        1720
A66 - 12 Recepció (2-1) Door #1           95
A66 - 17 Recepció (3-1) Door #1           74
A66 - 06 Kávézó terasz (F-3) Door #1      29
A66 - 18 Elõtér 316 (3-2) Door #1         29
A66 - 05 Vasrács (F-2) Door #1             6
A66 - 07 Recepció (1-1) Door #1            1
Name: Description #1, dtype: int64

#### Only consider log records from the front (df_regular_come is the main df)

In [25]:
result_cols = ['avg_arrival', 'median_arrival', 'deviation_arrival', 'avg_mins', 
              'longest_late', 'avg_late', 'go_out_freq_max', 'longest_late_day',
              'has_late_people', 'avg_to_2', 'avg_to_3']
terminate_name = []

In [26]:
doors

array(['A66 - 04 FÕBEJÁRAT (F-1) Door #1',
       'A66 - 12 Recepció (2-1) Door #1',
       'A66 - 17 Recepció (3-1) Door #1',
       'A66 - 06 Kávézó terasz (F-3) Door #1',
       'A66 - 18 Elõtér 316 (3-2) Door #1'], dtype='<U36')

In [27]:
df_front_door = df_regular_come_first[df_regular_come_first['Description #1'] == doors[0]]

In [28]:
df_front_door.head()

,Date time,date,time,Description #1,Card number,Description #2,Event message,day
0,2019-01-14 07:58:17,2019-01-14,07:58:17,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00070:04181,Barczi Mihály,Access granted,0
1,2019-01-14 08:26:32,2019-01-14,08:26:32,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00018:06288,Barish Nilofer Virág,Access granted,0
2,2019-01-14 08:27:48,2019-01-14,08:27:48,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00243:07476,Barta Gergõ,Access granted,0
3,2019-01-14 08:28:26,2019-01-14,08:28:26,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00178:59777,Takács Róbert,Access granted,0
4,2019-01-14 08:29:22,2019-01-14,08:29:22,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00161:44433,Salfay Anna,Access granted,0


In [29]:
def get_avg_arrival(df):
    return str(pd.to_timedelta(df_front_door['time']).mean().round('1s')).split(' ')[2]

get_avg_time(df_front_door)

'09:09:47'

In [45]:
def get_median_arrival(df):
    return df.sort_values(by= 'time').loc[df.shape[0] // 2]['time'] 

get_median_arrival(df_front_door)

'08:41:02'

In [30]:
# 1. The average arrival time
avg_arrival = get_avg_arrival(df_front_door)

In [ ]:
# 2. The median of arrival time
median_arrival = get_median_arrival(df_front_door)

In [304]:
df_door_21 = df_regular_come[df_regular_come['Description #1'] == doors[1]][sub_cols]
pd.Series(df_door_21['Card number'].value_counts() > 14).value_counts()

False    54
True     52
Name: Card number, dtype: int64

In [308]:
df_door_31 = df_regular_come[df_regular_come['Description #1'] == doors[2]][sub_cols]
pd.Series(df_door_31['Card number'].value_counts() > 14).value_counts()

False    97
True     38
Name: Card number, dtype: int64

In [309]:
df_door_F3 = df_regular_come[df_regular_come['Description #1'] == doors[3]][sub_cols]
pd.Series(df_door_F3['Card number'].value_counts() > 14).value_counts()

False    32
True     23
Name: Card number, dtype: int64

In [310]:
df_door_32 = df_regular_come[df_regular_come['Description #1'] == doors[4]][sub_cols]
pd.Series(df_door_32['Card number'].value_counts() > 14).value_counts()

False    58
True     11
Name: Card number, dtype: int64

In [58]:
"""
id highest frequence apperance in reception 2
"""
count = 5
for card_id in df_door_21['Card number'].value_counts().index:
    df_a_person = df_first_entry[df_first_entry['Card number'] == card_id]
    df_person_history = pd.merge(df_a_person,
                                df_week_day, on='date', how='left')
    display(df_person_history)
    count -= 1
    if count <= 0:
        break

,Date time,date,time,Description #1,Card number,Description #2,day
0,2019-01-07 09:04:57,2019-01-07,09:04:57,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00084:21322,Nagy Márton,0
1,2019-01-08 09:07:04,2019-01-08,09:07:04,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00084:21322,Nagy Márton,1
2,2019-01-09 10:02:28,2019-01-09,10:02:28,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00084:21322,Nagy Márton,2
3,2019-01-10 09:15:05,2019-01-10,09:15:05,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00084:21322,Nagy Márton,3
4,2019-01-11 08:37:17,2019-01-11,08:37:17,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00084:21322,Nagy Márton,4
5,2019-01-14 08:45:34,2019-01-14,08:45:34,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00084:21322,Nagy Márton,0
6,2019-01-15 08:51:39,2019-01-15,08:51:39,A66 - 12 Recepció (2-1) Door #1,00084:21322,Nagy Márton,1
7,2019-01-17 09:22:14,2019-01-17,09:22:14,A66 - 12 Recepció (2-1) Door #1,00084:21322,Nagy Márton,3
8,2019-01-18 10:05:14,2019-01-18,10:05:14,A66 - 12 Recepció (2-1) Door #1,00084:21322,Nagy Márton,4
9,2019-01-19 15:04:39,2019-01-19,15:04:39,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00084:21322,Nagy Márton,5


,Date time,date,time,Description #1,Card number,Description #2,day
0,2019-01-04 18:16:33,2019-01-04,18:16:33,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00214:48881,Morvay Gergely,4
1,2019-01-07 09:01:39,2019-01-07,09:01:39,A66 - 17 Recepció (3-1) Door #1,00214:48881,Morvay Gergely,0
2,2019-01-08 09:04:34,2019-01-08,09:04:34,A66 - 17 Recepció (3-1) Door #1,00214:48881,Morvay Gergely,1
3,2019-01-09 09:09:01,2019-01-09,09:09:01,A66 - 17 Recepció (3-1) Door #1,00214:48881,Morvay Gergely,2
4,2019-01-10 09:08:10,2019-01-10,09:08:10,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00214:48881,Morvay Gergely,3
5,2019-01-11 09:00:14,2019-01-11,09:00:14,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00214:48881,Morvay Gergely,4
6,2019-01-14 09:00:51,2019-01-14,09:00:51,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00214:48881,Morvay Gergely,0
7,2019-01-15 09:20:11,2019-01-15,09:20:11,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00214:48881,Morvay Gergely,1
8,2019-01-16 17:01:28,2019-01-16,17:01:28,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00214:48881,Morvay Gergely,2
9,2019-01-17 09:07:28,2019-01-17,09:07:28,A66 - 12 Recepció (2-1) Door #1,00214:48881,Morvay Gergely,3


,Date time,date,time,Description #1,Card number,Description #2,day
0,2019-01-02 09:29:15,2019-01-02,09:29:15,A66 - 12 Recepció (2-1) Door #1,00059:58046,Puskás Nóra,2
1,2019-01-03 13:36:44,2019-01-03,13:36:44,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00059:58046,Puskás Nóra,3
2,2019-01-04 09:28:18,2019-01-04,09:28:18,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00059:58046,Puskás Nóra,4
3,2019-01-07 09:22:41,2019-01-07,09:22:41,A66 - 17 Recepció (3-1) Door #1,00059:58046,Puskás Nóra,0
4,2019-01-08 09:10:38,2019-01-08,09:10:38,A66 - 12 Recepció (2-1) Door #1,00059:58046,Puskás Nóra,1
5,2019-01-10 16:35:02,2019-01-10,16:35:02,A66 - 17 Recepció (3-1) Door #1,00059:58046,Puskás Nóra,3
6,2019-01-14 11:14:30,2019-01-14,11:14:30,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00059:58046,Puskás Nóra,0
7,2019-01-15 11:19:30,2019-01-15,11:19:30,A66 - 17 Recepció (3-1) Door #1,00059:58046,Puskás Nóra,1
8,2019-01-16 09:20:23,2019-01-16,09:20:23,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00059:58046,Puskás Nóra,2
9,2019-01-17 13:12:48,2019-01-17,13:12:48,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00059:58046,Puskás Nóra,3


,Date time,date,time,Description #1,Card number,Description #2,day
0,2019-01-03 09:26:58,2019-01-03,09:26:58,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00111:27105,Makovics Ákos,3
1,2019-01-04 12:06:04,2019-01-04,12:06:04,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00111:27105,Makovics Ákos,4
2,2019-01-07 09:13:52,2019-01-07,09:13:52,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00111:27105,Makovics Ákos,0
3,2019-01-08 11:48:10,2019-01-08,11:48:10,A66 - 18 Elõtér 316 (3-2) Door #1,00111:27105,Makovics Ákos,1
4,2019-01-09 09:04:24,2019-01-09,09:04:24,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00111:27105,Makovics Ákos,2
5,2019-01-10 09:05:56,2019-01-10,09:05:56,A66 - 18 Elõtér 316 (3-2) Door #1,00111:27105,Makovics Ákos,3
6,2019-01-11 08:50:37,2019-01-11,08:50:37,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00111:27105,Makovics Ákos,4
7,2019-01-14 08:48:42,2019-01-14,08:48:42,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00111:27105,Makovics Ákos,0
8,2019-01-15 08:36:19,2019-01-15,08:36:19,A66 - 12 Recepció (2-1) Door #1,00111:27105,Makovics Ákos,1
9,2019-01-16 09:03:55,2019-01-16,09:03:55,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00111:27105,Makovics Ákos,2


,Date time,date,time,Description #1,Card number,Description #2,day
0,2019-01-07 11:08:55,2019-01-07,11:08:55,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00131:58577,Markos Dániel,0
1,2019-01-08 08:53:18,2019-01-08,08:53:18,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00131:58577,Markos Dániel,1
2,2019-01-09 07:56:11,2019-01-09,07:56:11,A66 - 17 Recepció (3-1) Door #1,00131:58577,Markos Dániel,2
3,2019-01-10 08:59:03,2019-01-10,08:59:03,A66 - 17 Recepció (3-1) Door #1,00131:58577,Markos Dániel,3
4,2019-01-11 08:52:47,2019-01-11,08:52:47,A66 - 17 Recepció (3-1) Door #1,00131:58577,Markos Dániel,4
5,2019-01-14 08:41:19,2019-01-14,08:41:19,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00131:58577,Markos Dániel,0
6,2019-01-15 08:41:12,2019-01-15,08:41:12,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00131:58577,Markos Dániel,1
7,2019-01-16 08:59:11,2019-01-16,08:59:11,A66 - 17 Recepció (3-1) Door #1,00131:58577,Markos Dániel,2
8,2019-01-17 08:59:04,2019-01-17,08:59:04,A66 - 17 Recepció (3-1) Door #1,00131:58577,Markos Dániel,3
9,2019-01-18 08:49:49,2019-01-18,08:49:49,A66 - 17 Recepció (3-1) Door #1,00131:58577,Markos Dániel,4


In [138]:
df_logs_after_14 = df_logs[df_logs['date'] >= '2019-01-14']
df_logs_after_14[sub_cols]

,Date time,date,time,Description #1,Card number,Description #2
1831,2019-01-14 07:58:17,2019-01-14,07:58:17,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00070:04181,Barczi Mihály
1832,2019-01-14 07:59:49,2019-01-14,07:59:49,A66 - 12 Recepció (2-1) Door #1,00070:04181,Barczi Mihály
1833,2019-01-14 08:26:32,2019-01-14,08:26:32,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00018:06288,Barish Nilofer Virág
1834,2019-01-14 08:27:21,2019-01-14,08:27:21,A66 - 12 Recepció (2-1) Door #1,00018:06288,Barish Nilofer Virág
1835,2019-01-14 08:27:48,2019-01-14,08:27:48,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00243:07476,Barta Gergõ
1836,2019-01-14 08:28:26,2019-01-14,08:28:26,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00178:59777,Takács Róbert
1837,2019-01-14 08:29:22,2019-01-14,08:29:22,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00161:44433,Salfay Anna
1838,2019-01-14 08:29:51,2019-01-14,08:29:51,A66 - 18 Elõtér 316 (3-2) Door #1,00178:59777,Takács Róbert
1839,2019-01-14 08:30:04,2019-01-14,08:30:04,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00234:15510,Vadász Péter
1840,2019-01-14 08:31:08,2019-01-14,08:31:08,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00218:22387,Kovács Péter


In [161]:
df_logs_after_14['Card number'].value_counts().shape

(174,)

In [150]:
df_logs_door_21 = df_logs_after_14[df_logs_after_14['Description #1'] == 'A66 - 12 Recepció (2-1) Door #1'][sub_cols]
pd.Series(df_logs_door_21['Card number'].value_counts() > 14).value_counts()

False    66
True     52
Name: Card number, dtype: int64

In [172]:
df_logs_door_21_count13 = pd.Series(df_logs_door_21['Card number'].value_counts() > 20)
people_21 = df_logs_door_21_count13.index[df_logs_door_21_count13.values].values
len(people_21)

34

In [152]:
df_logs_door_31 = df_logs_after_14[df_logs_after_14['Description #1'] == 'A66 - 17 Recepció (3-1) Door #1'][sub_cols]
pd.Series(df_logs_door_31['Card number'].value_counts() > 14).value_counts()

False    115
True      39
Name: Card number, dtype: int64

In [173]:
df_logs_door_31_count13 = pd.Series(df_logs_door_31['Card number'].value_counts() > 20)
people_31 = df_logs_door_31_count13.index[df_logs_door_31_count13.values].values
len(people_31)

20

In [174]:
len(set(people_21) - set(people_31))

32

In [113]:
df_logs_door_32 = df_logs[df_logs['Description #1'] == 'A66 - 18 Elõtér 316 (3-2) Door #1'][sub_cols]
pd.Series(df_logs_door_32['Card number'].value_counts() > 13).value_counts()

False    64
True     19
Name: Card number, dtype: int64

In [114]:
df_logs_door_F3 = df_logs[df_logs['Description #1'] == 'A66 - 06 Kávézó terasz (F-3) Door #1'][sub_cols]
pd.Series(df_logs_door_F3['Card number'].value_counts() > 13).value_counts()

False    46
True     24
Name: Card number, dtype: int64

In [98]:
df_logs['Card number'].value_counts().shape

(194,)

In [63]:

count = 5
for card_id in df_logs_door_21['Card number'].value_counts().index:
    df_a_person = df_logs[df_logs['Card number'] == card_id][sub_cols]
    df_person_history = pd.merge(df_a_person,
                                df_week_day, on='date', how='left')
    display(df_person_history)
    count -= 1
    if count <= 0:
        break

,Date time,date,time,Description #1,Card number,Description #2,day
0,2019-01-07 10:30:41,2019-01-07,10:30:41,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00151:37267,Szõke Tamás,0
1,2019-01-07 13:43:22,2019-01-07,13:43:22,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00151:37267,Szõke Tamás,0
2,2019-01-08 11:40:02,2019-01-08,11:40:02,A66 - 18 Elõtér 316 (3-2) Door #1,00151:37267,Szõke Tamás,1
3,2019-01-08 14:06:28,2019-01-08,14:06:28,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00151:37267,Szõke Tamás,1
4,2019-01-08 14:08:18,2019-01-08,14:08:18,A66 - 18 Elõtér 316 (3-2) Door #1,00151:37267,Szõke Tamás,1
5,2019-01-08 14:58:29,2019-01-08,14:58:29,A66 - 18 Elõtér 316 (3-2) Door #1,00151:37267,Szõke Tamás,1
6,2019-01-08 15:38:22,2019-01-08,15:38:22,A66 - 12 Recepció (2-1) Door #1,00151:37267,Szõke Tamás,1
7,2019-01-08 15:38:31,2019-01-08,15:38:31,A66 - 12 Recepció (2-1) Door #1,00151:37267,Szõke Tamás,1
8,2019-01-08 15:50:44,2019-01-08,15:50:44,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00151:37267,Szõke Tamás,1
9,2019-01-08 15:52:31,2019-01-08,15:52:31,A66 - 18 Elõtér 316 (3-2) Door #1,00151:37267,Szõke Tamás,1


,Date time,date,time,Description #1,Card number,Description #2,day
0,2019-01-03 09:26:58,2019-01-03,09:26:58,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00111:27105,Makovics Ákos,3
1,2019-01-03 09:28:10,2019-01-03,09:28:10,A66 - 18 Elõtér 316 (3-2) Door #1,00111:27105,Makovics Ákos,3
2,2019-01-03 10:38:29,2019-01-03,10:38:29,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00111:27105,Makovics Ákos,3
3,2019-01-03 10:40:21,2019-01-03,10:40:21,A66 - 18 Elõtér 316 (3-2) Door #1,00111:27105,Makovics Ákos,3
4,2019-01-03 15:30:45,2019-01-03,15:30:45,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00111:27105,Makovics Ákos,3
5,2019-01-03 15:32:01,2019-01-03,15:32:01,A66 - 18 Elõtér 316 (3-2) Door #1,00111:27105,Makovics Ákos,3
6,2019-01-03 17:02:34,2019-01-03,17:02:34,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00111:27105,Makovics Ákos,3
7,2019-01-03 17:03:58,2019-01-03,17:03:58,A66 - 18 Elõtér 316 (3-2) Door #1,00111:27105,Makovics Ákos,3
8,2019-01-04 12:06:04,2019-01-04,12:06:04,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00111:27105,Makovics Ákos,4
9,2019-01-04 12:07:11,2019-01-04,12:07:11,A66 - 18 Elõtér 316 (3-2) Door #1,00111:27105,Makovics Ákos,4


,Date time,date,time,Description #1,Card number,Description #2,day
0,2019-01-03 10:07:54,2019-01-03,10:07:54,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:42865,Szél Márton,3
1,2019-01-03 10:09:08,2019-01-03,10:09:08,A66 - 17 Recepció (3-1) Door #1,00110:42865,Szél Márton,3
2,2019-01-03 12:32:39,2019-01-03,12:32:39,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:42865,Szél Márton,3
3,2019-01-03 12:33:49,2019-01-03,12:33:49,A66 - 17 Recepció (3-1) Door #1,00110:42865,Szél Márton,3
4,2019-01-03 15:37:28,2019-01-03,15:37:28,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:42865,Szél Márton,3
5,2019-01-03 15:38:34,2019-01-03,15:38:34,A66 - 17 Recepció (3-1) Door #1,00110:42865,Szél Márton,3
6,2019-01-04 10:16:06,2019-01-04,10:16:06,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:42865,Szél Márton,4
7,2019-01-04 10:17:16,2019-01-04,10:17:16,A66 - 17 Recepció (3-1) Door #1,00110:42865,Szél Márton,4
8,2019-01-04 10:25:19,2019-01-04,10:25:19,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:42865,Szél Márton,4
9,2019-01-04 10:26:25,2019-01-04,10:26:25,A66 - 17 Recepció (3-1) Door #1,00110:42865,Szél Márton,4


,Date time,date,time,Description #1,Card number,Description #2,day
0,2019-01-07 09:18:43,2019-01-07,09:18:43,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00088:54928,Gulyás Edit,0
1,2019-01-07 10:52:51,2019-01-07,10:52:51,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00088:54928,Gulyás Edit,0
2,2019-01-07 11:44:25,2019-01-07,11:44:25,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00088:54928,Gulyás Edit,0
3,2019-01-07 11:46:11,2019-01-07,11:46:11,A66 - 17 Recepció (3-1) Door #1,00088:54928,Gulyás Edit,0
4,2019-01-07 12:37:56,2019-01-07,12:37:56,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00088:54928,Gulyás Edit,0
5,2019-01-07 12:39:08,2019-01-07,12:39:08,A66 - 17 Recepció (3-1) Door #1,00088:54928,Gulyás Edit,0
6,2019-01-07 13:59:28,2019-01-07,13:59:28,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00088:54928,Gulyás Edit,0
7,2019-01-07 14:01:27,2019-01-07,14:01:27,A66 - 17 Recepció (3-1) Door #1,00088:54928,Gulyás Edit,0
8,2019-01-07 15:01:57,2019-01-07,15:01:57,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00088:54928,Gulyás Edit,0
9,2019-01-07 17:21:27,2019-01-07,17:21:27,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00088:54928,Gulyás Edit,0


,Date time,date,time,Description #1,Card number,Description #2,day
0,2019-01-07 09:02:24,2019-01-07,09:02:24,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00182:64254,Berényi Gergely,0
1,2019-01-07 09:03:28,2019-01-07,09:03:28,A66 - 18 Elõtér 316 (3-2) Door #1,00182:64254,Berényi Gergely,0
2,2019-01-07 12:29:06,2019-01-07,12:29:06,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00182:64254,Berényi Gergely,0
3,2019-01-07 12:30:21,2019-01-07,12:30:21,A66 - 17 Recepció (3-1) Door #1,00182:64254,Berényi Gergely,0
4,2019-01-07 18:20:03,2019-01-07,18:20:03,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00182:64254,Berényi Gergely,0
5,2019-01-07 18:21:14,2019-01-07,18:21:14,A66 - 17 Recepció (3-1) Door #1,00182:64254,Berényi Gergely,0
6,2019-01-08 09:02:04,2019-01-08,09:02:04,A66 - 18 Elõtér 316 (3-2) Door #1,00182:64254,Berényi Gergely,1
7,2019-01-08 12:40:57,2019-01-08,12:40:57,A66 - 18 Elõtér 316 (3-2) Door #1,00182:64254,Berényi Gergely,1
8,2019-01-09 09:01:57,2019-01-09,09:01:57,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00182:64254,Berényi Gergely,2
9,2019-01-09 09:03:02,2019-01-09,09:03:02,A66 - 18 Elõtér 316 (3-2) Door #1,00182:64254,Berényi Gergely,2


In [236]:
"""
Get the least log frequency card number
"""
count = 20
for card_id in df_logs_after_14['Card number'].value_counts().index.values[::-1]:
    df_a_person = df_logs_after_14[df_logs_after_14['Card number'] == card_id][sub_cols]
    df_person_history = pd.merge(df_a_person,
                                df_week_day, on='date', how='left')
    display(df_person_history)
    count -= 1
    if count <= 0:
        break

,Date time,date,time,Description #1,Card number,Description #2,Event message,day
0,2019-01-18 15:03:44,2019-01-18,15:03:44,A66 - 17 Recepció (3-1) Door #1,00088:32688,Bóta Eszter,Access granted,4


,Date time,date,time,Description #1,Card number,Description #2,Event message,day
0,2019-01-17 17:13:43,2019-01-17,17:13:43,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00088:43952,Huszár Zsófia,Access granted,3


,Date time,date,time,Description #1,Card number,Description #2,Event message,day
0,2019-01-16 18:03:21,2019-01-16,18:03:21,A66 - 17 Recepció (3-1) Door #1,00039:59313,Nagy Krisztina Ágnes,Access granted,2


,Date time,date,time,Description #1,Card number,Description #2,Event message,day
0,2019-01-24 18:04:03,2019-01-24,18:04:03,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00155:46240,Bóta Ádám,Access granted,3


,Date time,date,time,Description #1,Card number,Description #2,Event message,day
0,2019-01-16 17:41:35,2019-01-16,17:41:35,A66 - 12 Recepció (2-1) Door #1,00078:08286,Ignácz Iringó,Access granted,2
1,2019-01-16 17:44:40,2019-01-16,17:44:40,A66 - 17 Recepció (3-1) Door #1,00078:08286,Ignácz Iringó,Access granted,2


...

,Date time,date,time,Description #1,Card number,Description #2,Event message,day
0,2019-01-24 10:43:37,2019-01-24,10:43:37,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00247:27091,Papp Dániel,Access granted,3
1,2019-01-24 12:56:37,2019-01-24,12:56:37,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00247:27091,Papp Dániel,Access granted,3
2,2019-01-24 14:35:13,2019-01-24,14:35:13,A66 - 06 Kávézó terasz (F-3) Door #1,00247:27091,Papp Dániel,Access granted,3
3,2019-01-24 17:55:17,2019-01-24,17:55:17,A66 - 06 Kávézó terasz (F-3) Door #1,00247:27091,Papp Dániel,Access granted,3
4,2019-01-24 18:21:37,2019-01-24,18:21:37,A66 - 17 Recepció (3-1) Door #1,00247:27091,Papp Dániel,Access granted,3
5,2019-01-25 09:13:45,2019-01-25,09:13:45,A66 - 06 Kávézó terasz (F-3) Door #1,00247:27091,Papp Dániel,Access granted,4
6,2019-01-25 12:54:52,2019-01-25,12:54:52,A66 - 06 Kávézó terasz (F-3) Door #1,00247:27091,Papp Dániel,Access granted,4
7,2019-01-25 15:18:31,2019-01-25,15:18:31,A66 - 06 Kávézó terasz (F-3) Door #1,00247:27091,Papp Dániel,Access granted,4


,Date time,date,time,Description #1,Card number,Description #2,Event message,day
0,2019-01-14 14:48:26,2019-01-14,14:48:26,A66 - 17 Recepció (3-1) Door #1,00039:27569,Tóth Szilvia,Access granted,0
1,2019-01-15 13:08:22,2019-01-15,13:08:22,A66 - 17 Recepció (3-1) Door #1,00039:27569,Tóth Szilvia,Access granted,1
2,2019-01-15 15:32:49,2019-01-15,15:32:49,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00039:27569,Tóth Szilvia,Access granted,1
3,2019-01-15 15:34:03,2019-01-15,15:34:03,A66 - 17 Recepció (3-1) Door #1,00039:27569,Tóth Szilvia,Access granted,1
4,2019-01-16 13:35:02,2019-01-16,13:35:02,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00039:27569,Tóth Szilvia,Access granted,2
5,2019-01-16 13:36:13,2019-01-16,13:36:13,A66 - 17 Recepció (3-1) Door #1,00039:27569,Tóth Szilvia,Access granted,2
6,2019-01-28 16:36:50,2019-01-28,16:36:50,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00039:27569,Tóth Szilvia,Access granted,0
7,2019-01-28 16:38:00,2019-01-28,16:38:00,A66 - 17 Recepció (3-1) Door #1,00039:27569,Tóth Szilvia,Access granted,0


,Date time,date,time,Description #1,Card number,Description #2,Event message,day
0,2019-01-16 18:00:12,2019-01-16,18:00:12,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00100:13024,Daru László,Access granted,2
1,2019-01-22 10:59:53,2019-01-22,10:59:53,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00100:13024,Daru László,Access granted,1
2,2019-01-24 11:12:26,2019-01-24,11:12:26,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00100:13024,Daru László,Access granted,3
3,2019-01-24 14:28:50,2019-01-24,14:28:50,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00100:13024,Daru László,Access granted,3
4,2019-01-24 18:30:46,2019-01-24,18:30:46,A66 - 06 Kávézó terasz (F-3) Door #1,00100:13024,Daru László,Access granted,3
5,2019-01-25 10:20:15,2019-01-25,10:20:15,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00100:13024,Daru László,Access granted,4
6,2019-01-25 13:13:04,2019-01-25,13:13:04,A66 - 06 Kávézó terasz (F-3) Door #1,00100:13024,Daru László,Access granted,4
7,2019-01-25 14:52:01,2019-01-25,14:52:01,A66 - 06 Kávézó terasz (F-3) Door #1,00100:13024,Daru László,Access granted,4


,Date time,date,time,Description #1,Card number,Description #2,Event message,day
0,2019-01-16 17:42:21,2019-01-16,17:42:21,A66 - 17 Recepció (3-1) Door #1,00036:38256,Balázs Eszter,Access granted,2
1,2019-01-24 10:12:15,2019-01-24,10:12:15,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00036:38256,Balázs Eszter,Access granted,3
2,2019-01-24 12:39:26,2019-01-24,12:39:26,A66 - 06 Kávézó terasz (F-3) Door #1,00036:38256,Balázs Eszter,Access granted,3
3,2019-01-24 12:39:32,2019-01-24,12:39:32,A66 - 06 Kávézó terasz (F-3) Door #1,00036:38256,Balázs Eszter,Access granted,3
4,2019-01-24 14:41:49,2019-01-24,14:41:49,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00036:38256,Balázs Eszter,Access granted,3
5,2019-01-25 10:03:52,2019-01-25,10:03:52,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00036:38256,Balázs Eszter,Access granted,4
6,2019-01-25 13:21:00,2019-01-25,13:21:00,A66 - 06 Kávézó terasz (F-3) Door #1,00036:38256,Balázs Eszter,Access granted,4
7,2019-01-25 15:02:52,2019-01-25,15:02:52,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00036:38256,Balázs Eszter,Access granted,4


,Date time,date,time,Description #1,Card number,Description #2,Event message,day
0,2019-01-22 11:00:45,2019-01-22,11:00:45,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00019:29239,Jákói Dávid,Access granted,1
1,2019-01-22 11:01:32,2019-01-22,11:01:32,A66 - 17 Recepció (3-1) Door #1,00019:29239,Jákói Dávid,Access granted,1
2,2019-01-24 10:11:20,2019-01-24,10:11:20,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00019:29239,Jákói Dávid,Access granted,3
3,2019-01-24 11:09:11,2019-01-24,11:09:11,A66 - 12 Recepció (2-1) Door #1,00019:29239,Jákói Dávid,Access granted,3
4,2019-01-24 12:39:37,2019-01-24,12:39:37,A66 - 12 Recepció (2-1) Door #1,00019:29239,Jákói Dávid,Access granted,3
5,2019-01-24 14:38:18,2019-01-24,14:38:18,A66 - 12 Recepció (2-1) Door #1,00019:29239,Jákói Dávid,Access granted,3
6,2019-01-24 14:49:37,2019-01-24,14:49:37,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00019:29239,Jákói Dávid,Access granted,3
7,2019-01-24 16:12:45,2019-01-24,16:12:45,A66 - 13 Elõtér 216 (2-2) Door #1,00019:29239,Jákói Dávid,Access granted,3
8,2019-01-24 18:01:36,2019-01-24,18:01:36,A66 - 13 Elõtér 216 (2-2) Door #1,00019:29239,Jákói Dávid,Access granted,3
9,2019-01-25 10:32:40,2019-01-25,10:32:40,A66 - 13 Elõtér 216 (2-2) Door #1,00019:29239,Jákói Dávid,Access granted,4


In [166]:
df_logs[df_logs['Card number'] == '00047:15422'][sub_cols]

,Date time,date,time,Description #1,Card number,Description #2
671,2019-01-08 11:52:48,2019-01-08,11:52:48,A66 - 12 Recepció (2-1) Door #1,00047:15422,Cserjés András
1122,2019-01-09 16:36:29,2019-01-09,16:36:29,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00047:15422,Cserjés András
1123,2019-01-09 16:37:01,2019-01-09,16:37:01,A66 - 12 Recepció (2-1) Door #1,00047:15422,Cserjés András
6587,2019-01-23 13:35:59,2019-01-23,13:35:59,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00047:15422,Cserjés András
6589,2019-01-23 13:36:30,2019-01-23,13:36:30,A66 - 12 Recepció (2-1) Door #1,00047:15422,Cserjés András
9328,2019-01-29 18:12:19,2019-01-29,18:12:19,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00047:15422,Cserjés András
9329,2019-01-29 18:13:06,2019-01-29,18:13:06,A66 - 12 Recepció (2-1) Door #1,00047:15422,Cserjés András


In [434]:
df_first_entry[df_first_entry['Card number'].isin(df_door_21['Card number'])]

,Date time,date,time,Description #1,Card number,Description #2
2,2019-01-02 09:29:15,2019-01-02,09:29:15,A66 - 12 Recepció (2-1) Door #1,00059:58046,Puskás Nóra
4,2019-01-02 09:36:51,2019-01-02,09:36:51,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter
6,2019-01-02 09:44:46,2019-01-02,09:44:46,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00008:58673,Szívós István
12,2019-01-02 10:59:54,2019-01-02,10:59:54,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00249:15139,Noszka András
38,2019-01-02 14:48:47,2019-01-02,14:48:47,A66 - 12 Recepció (2-1) Door #1,00088:60976,Horváth Balázs
56,2019-01-03 09:26:58,2019-01-03,09:26:58,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00111:27105,Makovics Ákos
62,2019-01-03 10:07:54,2019-01-03,10:07:54,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:42865,Szél Márton
64,2019-01-03 10:09:26,2019-01-03,10:09:26,A66 - 12 Recepció (2-1) Door #1,00088:60976,Horváth Balázs
65,2019-01-03 10:11:16,2019-01-03,10:11:16,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00066:49877,Császári Noémi
67,2019-01-03 10:15:18,2019-01-03,10:15:18,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00062:19216,Veres Attila


In [433]:
df_first_entry[df_first_entry['Card number'] == '00059:58046']

,Date time,date,time,Description #1,Card number,Description #2
2,2019-01-02 09:29:15,2019-01-02,09:29:15,A66 - 12 Recepció (2-1) Door #1,00059:58046,Puskás Nóra
102,2019-01-03 13:36:44,2019-01-03,13:36:44,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00059:58046,Puskás Nóra
142,2019-01-04 09:28:18,2019-01-04,09:28:18,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00059:58046,Puskás Nóra
356,2019-01-07 09:22:41,2019-01-07,09:22:41,A66 - 17 Recepció (3-1) Door #1,00059:58046,Puskás Nóra
614,2019-01-08 09:10:38,2019-01-08,09:10:38,A66 - 12 Recepció (2-1) Door #1,00059:58046,Puskás Nóra
1478,2019-01-10 16:35:02,2019-01-10,16:35:02,A66 - 17 Recepció (3-1) Door #1,00059:58046,Puskás Nóra
1971,2019-01-14 11:14:30,2019-01-14,11:14:30,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00059:58046,Puskás Nóra
2637,2019-01-15 11:19:30,2019-01-15,11:19:30,A66 - 17 Recepció (3-1) Door #1,00059:58046,Puskás Nóra
3147,2019-01-16 09:20:23,2019-01-16,09:20:23,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00059:58046,Puskás Nóra
4009,2019-01-17 13:12:48,2019-01-17,13:12:48,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00059:58046,Puskás Nóra


In [443]:
df_first_entry['Card number'].value_counts()

00082:08465    23
00152:61253    23
00215:09895    22
00008:58673    22
00055:39162    22
00147:18727    22
00186:21741    22
00110:57041    22
00075:09712    21
00110:42865    21
00066:49877    21
00111:27105    21
00028:11997    21
00028:12253    21
00056:07178    20
00234:15510    20
00088:54928    20
00084:25281    20
00161:44433    20
00214:48881    20
00084:18417    20
00182:64254    20
00006:33914    20
00150:59349    19
00106:22001    19
00118:62699    19
00081:35713    19
00029:49531    19
00081:36737    19
00106:17905    19
               ..
00121:38462     3
00117:63277     3
00077:56528     3
00042:32417     2
00247:27091     2
00235:25731     2
00088:43952     2
00207:46123     2
00088:56736     2
00089:02240     2
00208:53393     2
00241:15811     2
00182:61534     2
00028:04717     2
00195:17723     2
00088:28624     2
00089:01168     2
00088:32688     1
00038:24225     1
00241:13523     1
00136:28288     1
00038:28945     1
00113:26779     1
00078:08286     1
00155:4624

In [437]:
df_first_entry[df_first_entry['Card number'] == '00110:57041']

,Date time,date,time,Description #1,Card number,Description #2
4,2019-01-02 09:36:51,2019-01-02,09:36:51,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter
78,2019-01-03 10:58:52,2019-01-03,10:58:52,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter
168,2019-01-04 12:12:40,2019-01-04,12:12:40,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter
257,2019-01-07 08:32:11,2019-01-07,08:32:11,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter
714,2019-01-08 12:52:23,2019-01-08,12:52:23,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter
906,2019-01-09 09:08:05,2019-01-09,09:08:05,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter
1238,2019-01-10 08:59:10,2019-01-10,08:59:10,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter
1574,2019-01-11 08:42:35,2019-01-11,08:42:35,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter
1910,2019-01-14 09:05:13,2019-01-14,09:05:13,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter
2485,2019-01-15 09:00:45,2019-01-15,09:00:45,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter


In [451]:
pd.merge(df_first_entry[df_first_entry['Card number'] == '00110:57041'], 
        df_week_day, on= 'date', how= 'left')

,Date time,date,time,Description #1,Card number,Description #2,day
0,2019-01-02 09:36:51,2019-01-02,09:36:51,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter,2
1,2019-01-03 10:58:52,2019-01-03,10:58:52,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter,3
2,2019-01-04 12:12:40,2019-01-04,12:12:40,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter,4
3,2019-01-07 08:32:11,2019-01-07,08:32:11,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter,0
4,2019-01-08 12:52:23,2019-01-08,12:52:23,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter,1
5,2019-01-09 09:08:05,2019-01-09,09:08:05,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter,2
6,2019-01-10 08:59:10,2019-01-10,08:59:10,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter,3
7,2019-01-11 08:42:35,2019-01-11,08:42:35,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter,4
8,2019-01-14 09:05:13,2019-01-14,09:05:13,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter,0
9,2019-01-15 09:00:45,2019-01-15,09:00:45,A66 - 04 FÕBEJÁRAT (F-1) Door #1,00110:57041,Ripka Péter,1


In [445]:
names = df_first_entry['Description #2'].values

In [448]:
set(names)

{'Antal Péter',
 'Aranyi István',
 'Bader Bettina',
 'Bak Roland',
 'Bakó Farkas',
 'Balogh Botond',
 'Balázs Eszter',
 'Baracs Áron',
 'Barczi Mihály',
 'Bardos Barbara',
 'Barish Nilofer Virág',
 'Barta Dávid',
 'Barta Gergõ',
 'Barta Viktor',
 'Bauer Dezsõ',
 'Bencz Dániel',
 'Bene Ádám',
 'Bengyel Gábor',
 'Berbardo Diana',
 'Berényi Gergely',
 'Bogár László',
 'Bonnyai-Pap Andrea',
 'Bori Kinga',
 'Buku Réka',
 'Bánszegi Bence',
 'Báthory Róbert',
 'Bátori Krisztián',
 'Bíró Adél',
 'Bíró Botond',
 'Bóta Eszter',
 'Bóta Ádám',
 'Bürg Ákos',
 'Cserjés András',
 'Csikós Zoltán',
 'Csima Gergely',
 'Császári Noémi',
 'Czakói Levente',
 'Czender András',
 'Daninger Viktória',
 'Daru László',
 'Deszpoth Csilla',
 'Deák Gabriella',
 'Dmytro Kopan',
 'Dobrozemszky Gábor',
 'Domány Juli',
 'Ecseki Tibor',
 'Egri Zoltán',
 'Farkas János',
 'Fodor Gábor',
 'Fodor Krisztián',
 'Formanek Dénes',
 'Frankó Zsuzsanna',
 'Fábián Zsófia Eszter',
 'Gerencsér András',
 'Grill Rudolf',
 'Gräfl András